In [ ]:
import os
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
import pickle
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
!pip install fuzzywuzzy
from fuzzywuzzy import process

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)  

This method uses ratings, movies and the tag file.

In [ ]:
df_ratings = pd.read_csv('/content/drive/My Drive/rating.csv', parse_dates=['timestamp'])
df_movies = pd.read_csv('/content/drive/My Drive/movie.csv')
df_tag = pd.read_csv('/content/drive/My Drive/tag.csv')

In [ ]:
# df_ratings_new = pd.read_pickle('/content/drive/My Drive/df_ratings_new.p')

In [ ]:
print(df_ratings_new)
print(df_movies)
print(df_tag)

          userId  movieId  rating           timestamp
0              1        2     3.5 2005-04-02 23:53:47
1              1       29     3.5 2005-04-02 23:31:16
2              1       32     3.5 2005-04-02 23:33:39
3              1       47     3.5 2005-04-02 23:32:07
4              1       50     3.5 2005-04-02 23:29:40
...          ...      ...     ...                 ...
20000258  138493    68954     4.5 2009-11-13 15:42:00
20000259  138493    69526     4.5 2009-12-03 18:31:48
20000260  138493    69644     3.0 2009-12-07 18:10:57
20000261  138493    70286     5.0 2009-11-13 15:42:24
20000262  138493    71619     2.5 2009-10-17 20:25:36

[16817989 rows x 4 columns]
       movieId  ...                                       genres
0            1  ...  Adventure|Animation|Children|Comedy|Fantasy
1            2  ...                   Adventure|Children|Fantasy
2            3  ...                               Comedy|Romance
3            4  ...                         Comedy|Drama|Romanc

Filters the data so that only ratings from users that have rated at least 80 movies are counted. This basically removes outliers and after testing, does not remove significant amounts of data.

In [ ]:
df_ratings_new = df_ratings.groupby('userId').filter(lambda movies_rated : len(movies_rated) > 80)
df_ratings_new.to_pickle("df_ratings_new.p")

Changes the movie dataframe to account for any movies that may not be rated in the rating matrix anymore

In [ ]:
included_movies = df_ratings_new.movieId.unique()
df_movies = df_movies[df_movies.movieId.isin(included_movies)]
print(df_movies)

       movieId  ...                                       genres
0            1  ...  Adventure|Animation|Children|Comedy|Fantasy
1            2  ...                   Adventure|Children|Fantasy
2            3  ...                               Comedy|Romance
3            4  ...                         Comedy|Drama|Romance
4            5  ...                                       Comedy
...        ...  ...                                          ...
27273   131254  ...                                       Comedy
27274   131256  ...                                       Comedy
27275   131258  ...                                    Adventure
27276   131260  ...                           (no genres listed)
27277   131262  ...                     Adventure|Fantasy|Horror

[26676 rows x 3 columns]


Dataframe is too large to create a pivot table so needs to be reduced to 1000000 rows and the dataframes are merged to then create a pivot table.

In [ ]:
df_ratings_new = df_ratings_new.iloc[:1000000,:]
df_merged = pd.merge(df_movies[['movieId']], df_ratings_new)
pt = df_merged.pivot(index='movieId', columns='userId', values='rating').fillna(0)
print(pt)

userId   1     3     7     11    14    18    ...  8086  8089  8090  8091  8092  8093
movieId                                      ...                                    
1         0.0   4.0   0.0   4.5   4.5   0.0  ...   0.0   0.0   0.0   0.0   4.0   0.0
2         3.5   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   1.0   4.0
3         0.0   0.0   3.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   1.0   0.0
4         0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
5         0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   2.0   0.0
...       ...   ...   ...   ...   ...   ...  ...   ...   ...   ...   ...   ...   ...
130219    0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
130462    0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
130490    0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
130496    0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.

Sparse matrix created using the pivot table

In [ ]:
movie_matrix = csr_matrix(pt.values)
print(movie_matrix)

  (0, 1)	4.0
  (0, 3)	4.5
  (0, 4)	4.5
  (0, 7)	3.0
  (0, 8)	4.0
  (0, 9)	4.0
  (0, 12)	3.0
  (0, 13)	5.0
  (0, 20)	4.0
  (0, 21)	4.0
  (0, 23)	5.0
  (0, 25)	4.0
  (0, 30)	5.0
  (0, 33)	3.5
  (0, 34)	4.0
  (0, 35)	3.5
  (0, 41)	3.5
  (0, 43)	3.0
  (0, 45)	5.0
  (0, 46)	4.0
  (0, 47)	3.0
  (0, 48)	2.0
  (0, 50)	4.0
  (0, 51)	4.0
  (0, 52)	4.0
  :	:
  (14223, 2642)	4.0
  (14224, 299)	0.5
  (14224, 2788)	0.5
  (14225, 1352)	2.5
  (14226, 1352)	3.0
  (14227, 2019)	2.0
  (14228, 493)	3.0
  (14229, 422)	4.0
  (14230, 2823)	1.0
  (14231, 2019)	2.0
  (14232, 2501)	3.5
  (14233, 1526)	2.5
  (14234, 2823)	3.5
  (14235, 1309)	3.5
  (14236, 1484)	3.5
  (14236, 2635)	3.0
  (14237, 3351)	3.0
  (14238, 1730)	1.0
  (14239, 2019)	2.5
  (14240, 580)	4.5
  (14241, 2592)	4.0
  (14242, 1087)	1.0
  (14242, 1526)	3.5
  (14243, 3303)	1.5
  (14244, 1730)	3.0


This function takes in a movie and an int n and gives back n movies that are similar to the movie using KNN.

In [ ]:
def knn_recommender(movie, n):
  classifier = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=30)
  classifier.fit(movie_matrix)
  mov_info = process.extractOne(movie, df_movies['title'])
  mov_i = mov_info[2]
  distances, items=classifier.kneighbors(movie_matrix[mov_i], n_neighbors=n)
  for i in items:
    print(df_movies['title'][i])

A brief example using Catwalk:

In [ ]:
knn_recommender('Catwalk (1996)', 20)

106                                        Catwalk (1996)
3557    Abominable Snowman, The (Abominable Snowman of...
7532                           Vampire Lovers, The (1970)
3279                          Devil's Brigade, The (1968)
4892                            White Water Summer (1987)
5675    Night of the Zombies (a.k.a. Batallion of the ...
5673                          My Dinner with André (1981)
3287                              Destination Moon (1950)
5278                             Harvey Girls, The (1946)
7590                                   Bloody Mama (1970)
7197                             Black Pirate, The (1926)
4942                        Flight of Dragons, The (1982)
5905                Abraham's Valley (Vale Abraão) (1993)
4896                                Kate & Leopold (2001)
735                                   Arrival, The (1996)
4933                                    China Moon (1994)
7144                             Touching the Void (2003)
5297          